In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.optimizers import SGD
import math
import pickle

Using TensorFlow backend.


In [2]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [3]:
X_train = dataset[0][0][0]["AAPL"].values
y_train = dataset[0][0][1]["AAPL"].values
X_test = dataset[1][0][0]["AAPL"].values
y_test = dataset[1][0][1]["AAPL"].values

In [4]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750,)
y train shape: (750,)
x test shape: (250,)
y test shape: (250,)


In [5]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [6]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [7]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [8]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1], 1))

In [9]:
X_train.shape

(510, 240, 1)

In [10]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 1)))
# The output layer
regressor.add(LSTM(32))
regressor.add(Dense(1, activation='softmax'))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_crossentropy'])

# Fitting to the training set
regressor.fit(X_train,y_train,epochs=10,batch_size=10)

Epoch 1/10
510/510 [==============================] - 20s 38ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 2/10
510/510 [==============================] - 19s 37ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 3/10
510/510 [==============================] - 19s 37ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 4/10
510/510 [==============================] - 20s 39ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 5/10
510/510 [==============================] - 19s 37ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 6/10
510/510 [==============================] - 18s 36ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 7/10
510/510 [==============================] - 19s 37ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 8/10
510/510 [==============================] - 19s 38ms/step - loss: 7.6586 - binary_crossentropy: 7.6586
Epoch 9/10
510/510 [==============================] - 18s 36ms/step - loss: 7.6586 - binary_cros

In [11]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [12]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int
label

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [13]:
(sum(y_test == label)/len(y_test)).mean()

0.7